In [ ]:
import numpy as np
import kwant
import phlead
from scipy.constants import hbar, m_e, eV, physical_constants
import matplotlib.pyplot as plt


meV = eV * 1e-3  # one meV in J
mu_B = physical_constants['Bohr magneton'][0] / meV  # Bohr magneton in meV
g = 50
m = 0.015 * m_e  # the effective mass in kg
a = 5  # the lattice spacing in nm
t = (hbar ** 2 / (2 * m * a ** 2)) * (1e18 / meV)  # meV

s0 = np.array([[1., 0.], [0., 1.]])
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1j], [1j, 0.]])
sz = np.array([[1., 0.], [0., -1.]])

# Tunnel spectroscopy
# the system will consist of three subsystems
def make_system_spectroscopy(t=t, a=a):

    # We apply a magnetic field in all parts of the system
    def onsite_sc(site, mu_l, mu_sc, B_x, alpha, delta, Vbarrier):
        return (2 * t - mu_sc) * np.kron(s0, sz) + (0.5 * B_x * g * mu_B) * np.kron(sx, s0) + delta * np.kron(s0, sx)

    def onsite_normal(site, mu_l, mu_sc, B_x, alpha, delta, Vbarrier):
        return (2 * t - mu_l) * np.kron(s0, sz) + (0.5 * B_x * g * mu_B) * np.kron(sx, s0)

    def onsite_barrier(site, mu_l, mu_sc, B_x, alpha, delta, Vbarrier):
        return (2 * t - mu_l + Vbarrier) * np.kron(s0, sz) + (0.5 * B_x * g * mu_B) * np.kron(sx, s0)

    # The hopping is the same in all subsystems. There is normal hopping and
    # spin orbit interaction.
    def hop(site1, site2, mu_l, mu_sc, B_x, alpha, delta, Vbarrier):
        return -t * np.kron(s0, sz) + 0.5j * alpha / a * np.kron(sy, sz)

    lat = kwant.lattice.chain(a=a)
    sys = kwant.Builder()

    # The first subsystem just consists of the tunnel barrier (one site with a
    # potential)
    sys[lat(0)] = onsite_barrier

    # The second subsystem is a normal lead
    # The translational symmetry makes it semiinfinite.
    symmetry = kwant.TranslationalSymmetry((-a,))
    lead_normal = kwant.Builder(symmetry)
    # Define a unit cell (in this case the unitcell consists of a single site)
    lead_normal[lat(0)] = onsite_normal
    # Define the hopping between unitcells
    lead_normal[lat(1), lat(0)] = hop

    # The third subsystem is a superconducting lead. A Majorana bound state
    # can arise at the edge of this system.
    lead_sc = kwant.Builder(symmetry)
    # Again: Define a unitcell
    lead_sc[lat(0)] = onsite_sc
    # Again define hopping between unitcells
    lead_sc[lat(1), lat(0)] = hop

    # Create a connection between the first subsystem (the tunnelbarrier, system name: sys)
    # and the other two subsystems (the normal and the superconducting lead)
    sys.attach_lead(lead_normal)
    sys.attach_lead(lead_sc)

    sys = sys.finalized()

    # This line of code is necessary to be able to calculate Andreev reflection.
    # The praticle-hole symmetry operator is explicitly passed to this
    # function.
    phlead.make_ph_lead(sys, 0, np.kron(sy, sy))

    return sys


def tunnel_spectroscopy(mu_l, mu_sc, B_x, alpha, delta, Vbarrier):
    sys = make_system_spectroscopy()
    Es = np.linspace(-1.2*delta, 1.2*delta, 51) + 1e-5
    Gs = []
    for E in Es:
        # This function returns the scatering matrix (at the energy given by the second argument)
        # of the system. The scattering matrix holds the information about
        # conductivity.
        smat = kwant.smatrix(
            sys, energy=E, args=(mu_l, mu_sc, B_x, alpha, delta, Vbarrier))
        Gs.append(phlead.conductance(smat, 0))

    return Es, Gs


def plot_spectroscopy(mu_l, mu_sc, B_x, alpha, delta, Vbarrier):
    fig, ax = plt.subplots()

    # Trivial, because the magnetic field is zero (third argument)
    Es, Gs = tunnel_spectroscopy(mu_l, mu_sc, 0, alpha, delta, Vbarrier)
    ax.plot(Es, Gs, 'k-', label='trivial')

    # Non-trivial
    Es, Gs = tunnel_spectroscopy(mu_l, mu_sc, B_x, alpha, delta, Vbarrier)
    ax.plot(Es, Gs, 'r-', label='non trivial')

    ax.set_title('Tunnel spectroscopy')
    ax.set_xlabel('Bias Voltage')
    ax.set_xlim([-1.2 * delta, 1.2 * delta])
    ax.set_ylabel('$G/G_0$')
    ax.set_ylim([-0.0, 2.1])

    ax.legend(fontsize=7)

    return fig

All units are in nm and meV.

In [ ]:
from ipywidgets import interact, FloatSlider
interact(plot_spectroscopy, 
         mu_l=FloatSlider(min=0, max=50, step=0.1, value=15, description=r"$\mu_l$ in meV"),
         mu_sc=FloatSlider(min=0, max=25, step=0.1, value=0, description=r"$\mu_{SC}$ in meV"), 
         B_x=FloatSlider(min=0, max=10, step=0.05, value=0.5, description=r"Magnetic field $(B)$ in T"),
         alpha=FloatSlider(min=0, max=50, step=0.5, value=20, description=r"SO coupling $(\alpha)$ in meV$\cdot$nm"),
         delta=FloatSlider(min=0, max=0.5, step=0.05, value=0.25, description=r"SC order param. ($\Delta$) in meV"),
         Vbarrier=FloatSlider(min=0, max=100, step=0.25, description=r"$V_{barrier}$ in meV"))